## Solving bidomain equations on rectangular domain

### Importing required libraries

In [ ]:
%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl

### Defining a domain, cell model and dynamics model and visualizing initial data

In [ ]:
domain = rectangle(4, 2, 100, 50)
x = ufl.SpatialCoordinate(domain)

In [ ]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85
        locator = lambda x: x[0] < 0.1
        cells = fem.locate_dofs_geometrical(self.V1, locator)
        self.V_m_n.x.array[cells] = np.full_like(cells, -50)

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector(
            [x[1] ** 2 / ufl.sqrt((x[1] ** 4 + 1)), 1 / ufl.sqrt((x[1] ** 4 + 1))]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IN - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EN - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

In [ ]:
cell_model = BeelerReuter(domain)
model = Model(domain, cell_model)

In [ ]:
model.plot_initial_V_m(camera_direction="xy", function_name="")

### Solving equations with given parameters

In [ ]:
model.solve(
    T=50,
    steps=1000,
    save_to="rectangle/bidomain_BR.mp4",
    signal_point=[1.0, 1.0, 0.0],
    camera_direction="xy",
    checkpoints=[10,20,30,40,50],
    checkpoint_file='figures/rectangle/bidomain_BR',
)

### Plotting a fiber orientations

In [ ]:
plot_vector_field(
    domain,
    lambda x: [x[1] ** 2 / np.sqrt((x[1] ** 4 + 1)), 1 / np.sqrt((x[1] ** 4 + 1)), 0],
    0.03,
    0.2,
    camera_direction="xy",
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()